In [7]:
import numpy as np
import pandas as pd
import scipy.stats as spstat
dataDir     = './data/'

import pandas as pd
#pack E field DF
# Manually defined headers based on the sample you provided
headers = [
    "roll (m)", "yaw (m)", "pitch (m)", "freq (MHz)", 
    "E_x",
    "E_y",
    "E_z"
]

# Read the file using the headers and consider multiple spaces as a delimiter
file_path = "eField_9_13_23.txt"
E_df = pd.read_csv(dataDir + file_path, delim_whitespace=True, names=headers, skiprows=5)

# Convert string representation of complex numbers to actual complex numbers
def to_complex(value):
    return complex(value.replace('i', 'j'))

columns_to_convert = headers[4:]  # Last three columns based on your sample

def to_complex(value):
    if isinstance(value, str) and ('i' in value or '+' in value or '-' in value):
        return complex(value.replace('i', 'j'))
    return value


# Convert strings to complex numbers
for col in columns_to_convert:
    E_df[col] = E_df[col].apply(to_complex)

# Convert complex numbers to their magnitudes
E_df[columns_to_convert] = E_df[columns_to_convert].applymap(abs)

In [8]:
print('E_x:')
print(E_df['E_x'].mean())
print(spstat.bootstrap((E_df['E_x'].to_numpy(),), np.mean, n_resamples= 9999))

print('E_y:')
print(E_df['E_y'].mean())
print(spstat.bootstrap((E_df['E_y'].to_numpy(),), np.mean, n_resamples= 9999))

print('E_z:')
print(E_df['E_z'].mean())
print(spstat.bootstrap((E_df['E_z'].to_numpy(),), np.mean, n_resamples= 9999))

E_x:
0.4713488213416364
BootstrapResult(confidence_interval=ConfidenceInterval(low=0.46382962486214935, high=0.47896779161108083), standard_error=0.003866721167286478)
E_y:
0.5395484396134266
BootstrapResult(confidence_interval=ConfidenceInterval(low=0.5317630663634096, high=0.5474575615927316), standard_error=0.0040090063154351345)
E_z:
0.40578469013230867
BootstrapResult(confidence_interval=ConfidenceInterval(low=0.3981526546229576, high=0.4135644331119586), standard_error=0.003893926587779368)


In [9]:
# Pack LPV DF

headers = [
    "roll (m)", "yaw (m)", "pitch (m)", "freq (MHz)", 
    'LPV'
]

# Read the file using the headers and consider multiple spaces as a delimiter
file_path = "LPV.txt"
lpv_df = pd.read_csv(dataDir + file_path, delim_whitespace=True, names=headers, skiprows=5)

1/lpv_df['LPV'].mean()

3.2634965601752994

In [10]:
data    = lpv_df['LPV'].to_numpy()
spstat.bootstrap((data,), np.mean, n_resamples= 9999)

BootstrapResult(confidence_interval=ConfidenceInterval(low=0.3018526151115846, high=0.31108642083799176), standard_error=0.0023453511620699606)

In [36]:
####Power and aperature####
print(f'Power from comsol (95% CL): \
{np.round(0.30185**2/50 *1000, 2)}-{np.round(0.31109**2/50 *1000,2)} mW'
)


lam             = 2.99e8/62.7e6  
Z_fs            = 376 # Z free space, Ohms
m               = 1 # match
e               = 1 # efficency
p               = .5 #polarization 
theoryPower     = 1**2 * lam**2/(4*np.pi*Z_fs) * m * e * p
print(f'Power from Hill eq 45: {np.round(theoryPower * 1000, 2)} mW')
print()

####Fields and AF####
print(f'AF from comsol (95% CL): \
{np.round(1/0.30185, 2)}-{np.round(1/0.31109,2)}'
)

print(f'AF from P * 9.7/lam  (95% CL): \
{np.round(np.sqrt(3)* 9.73/lam, 2)}'
)


Power from comsol (95% CL): 1.82-1.94 mW
Power from Hill eq 45: 2.41 mW

AF from comsol (95% CL): 3.31-3.21
AF from P * 9.7/lam  (95% CL): 3.53


In [37]:
3.25/3.53 #af comsol/AF therory

0.9206798866855525

In [35]:
1.87/2.41 #power comsol/power theory


0.7759336099585062